In [ ]:
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import LoraConfig, PeftModelForCausalLM
from datasets import load_from_disk

In [2]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16
)

In [3]:
base_model = AutoModel.from_pretrained('THUDM/chatglm2-6b', trust_remote_code=True, device_map='auto', quantization_config=quantization_config)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 7/7 [00:15<00:00,  2.26s/it]


In [4]:
base_model

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [5]:
tokenizer = AutoTokenizer.from_pretrained('THUDM/chatglm2-6b', trust_remote_code=True)

In [6]:
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [7]:
response, history = base_model.chat(tokenizer, '输出一份黄山旅游的计划', history=[])

In [8]:
response

'以下是一份黄山旅游计划:\n\n第一天:\n- 上午:到达黄山市,前往酒店办理入住手续并放下行李。\n- 下午:前往黄山市区,游览黄山市政府广场和黄山市博物馆,了解黄山市的旅游文化和历史。\n\n第二天:\n- 上午:前往黄山市区的黄山风景区,乘坐缆车上山。\n- 下午:游览西海大峡谷、天都峰等著名景点,欣赏黄山的美丽景色。\n\n第三天:\n- 上午:前往黄山市区的黄山市博物馆,了解黄山市的民俗文化和特色美食。\n- 下午:前往黄山市的徽州区,游览潜口民宅、呈坎古镇等景点,体验徽派建筑和文化。\n\n第四天:\n- 上午:前往黄山市的黄山区,游览黄山南屏景区、汤口镇等景点,了解黄山的自然风光和地理文化。\n- 下午:前往黄山市的黄山风景区,乘坐缆车上山。\n\n第五天:\n- 上午:游览黄山市的黄山风景区,欣赏黄山的美丽景色,乘坐缆车下山。\n- 下午:前往黄山市的市区,游览黄山市区的景点,如黄山市体育馆、黄山市图书馆等。\n\n第六天:\n- 上午:前往黄山市的机场,乘坐飞机返回原所在地。\n\n以上是一份五天四夜的黄山旅游计划,可以根据自己的需要和时间进行适当调整。希望这份计划能有所帮助!'

In [9]:
datasets = load_from_disk(dataset_path='data/Med_datasets.jsonl')

In [10]:
datasets['train'][1]

{'context': '指令：下面是一个医学领域的问题，请根据你对医学领域的了解，严谨细致的回答，如果不清楚就回答不清楚，不许胡编乱造。\n问题：患者孕14周，主诉右上腹部疼痛，伴随恶心、呕吐，BP 160/110 mmHg，体检腹肌紧张，有肝区压痛。请问可能患有什么疾病？\n回答: ',
 'target': '患者可能患有妊娠期高血压疾病引起的妊娠期肝血肿及破裂，需要进行B超检查、实验室检查等辅助检查，对症给予肝素、冻干人血浆等治疗方案。'}

In [11]:
query = datasets['train'][1]['context']

In [12]:
response, history = base_model.chat(tokenizer, query=query, history=[])

In [13]:
response

'根据病史和体检结果,患者可能患有妊娠期高血压疾病(简称妊娠期高血压)。妊娠期高血压疾病是指妊娠期间发生的高血压疾病,是孕期最常见的并发症之一。其特点是高血压、蛋白尿、胎盘早剥和宫内并发症等。\n\n患者出现右上腹部疼痛、恶心、呕吐、高血压、肝区压痛等症状,提示可能存在腹部疾病。而根据体检结果,腹肌紧张和肝区压痛进一步支持了这一诊断。\n\n因此,患者可能患有妊娠期高血压疾病,需要及时就医并接受医生的进一步检查和治疗。'

In [15]:
datasets['valid'][158]

{'context': '指令：下面是一个医学领域的问题，请根据你对医学领域的了解，严谨细致的回答，如果不清楚就回答不清楚，不许胡编乱造。\n问题：某女性患者出现敏感、水疱、皮损为对称性、水肿等症状，并伴有关节痛等症状，诊断结果为什么？\n回答: ',
 'target': '考虑为亚急性皮肤红斑狼疮，需进一步检查实验室检查如红细胞沉降率和免疫学检查，治疗方案可以包括组胺、氨苯砜、普鲁卡因、羟氯喹等。'}

In [16]:
query = datasets['valid'][158]['context']

In [17]:
response, history = base_model.chat(tokenizer, query=query, history=[])

In [18]:
response

'根据症状描述,该女性患者可能患有带状疱疹。带状疱疹是由水痘病毒引起的一种疾病,通常会在身体的一侧出现带状的皮疹、神经痛和局部肿胀等症状。如果病毒侵入到神经组织中,它可能会引起带状疱疹性神经痛,这种疼痛可能会持续数月。\n\n该女性的症状包括敏感、水疱、皮损为对称性、水肿和关节痛,这些症状与带状疱疹的特征相符合。此外,带状疱疹通常会在春季和秋季出现,因此这也是一个提示。\n\n综上所述,根据症状描述,我怀疑该女性患有带状疱疹。建议她立即就医,并进行病毒学检查以确认诊断,以便及时治疗。'

In [19]:
lora_config = LoraConfig.from_pretrained('Lora_Adapter_THUDM_chatglm2-6b/finally_adapter/LoraAdapter')

In [20]:
lora_config

LoraConfig(peft_type='LORA', base_model_name_or_path='THUDM/chatglm2-6b', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=12, target_modules=['query_key_value'], lora_alpha=24, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [21]:
model = PeftModelForCausalLM.from_pretrained(base_model, 'Lora_Adapter_THUDM_chatglm2-6b/finally_adapter/LoraAdapter', adapter_name='LoraAdapter')

In [22]:
model.base_model.model.transformer.encoder.layers[0].self_attention.query_key_value.lora_A.LoraAdapter.weight

Parameter containing:
tensor([[-0.0010, -0.0133, -0.0148,  ..., -0.0097, -0.0126,  0.0098],
        [-0.0066, -0.0118,  0.0112,  ..., -0.0013,  0.0085, -0.0175],
        [-0.0032,  0.0003,  0.0143,  ...,  0.0143,  0.0175,  0.0184],
        ...,
        [-0.0042,  0.0138, -0.0155,  ...,  0.0193,  0.0089,  0.0157],
        [ 0.0146,  0.0074,  0.0165,  ...,  0.0215, -0.0120,  0.0076],
        [ 0.0036, -0.0030,  0.0110,  ...,  0.0192, -0.0008,  0.0101]],
       device='cuda:0')

In [ ]:
query = datasets['train'][1]['context']

In [29]:
response, history = model.chat(tokenizer, query=query, history=[])

In [30]:
response

'该女性患有皮炎。'

In [ ]:
query = query = datasets['valid'][158]['context']

In [31]:
response, history = model.chat(tokenizer, query=query, history=[])

In [32]:
response

'该患者可能患有皮肌炎。'

In [33]:
query = "指令：下面是一个医学领域的问题，请根据你对医学领域的了解，严谨细致的回答，如果不清楚就回答不清楚，不许胡编乱造。\n问题：患者反复出现反酸、烧心等症状，考虑为Barrett食管，需要注意哪些并发症？\n回答: "

In [34]:
response, history = model.chat(tokenizer, query=query, history=[])

In [35]:
response

'需要注意并发症，包括食管穿孔、下食管括约肌不全、食管梗阻等。'

In [47]:
response, history = model.chat(tokenizer, '你会写python脚本吗', history=[])

In [48]:
response

'是的,我可以写Python脚本。Python是一种非常流行的编程语言,它的语法简洁易于理解,因此在各个领域中都有广泛的应用。你有什么特定的需求吗?我可以帮你写一个简单的Python脚本来帮助你开始。'

In [39]:
query = "指令：下面是一个医学领域的问题，请根据你对医学领域的了解，严谨细致的回答，如果不清楚就回答不清楚，不许胡编乱造。\n问题：患者反复出现反酸、烧心等症状，考虑为Barrett食管，需要注意哪些并发症？\n回答: "

In [40]:
query

'指令：下面是一个医学领域的问题，请根据你对医学领域的了解，严谨细致的回答，如果不清楚就回答不清楚，不许胡编乱造。\n问题：患者反复出现反酸、烧心等症状，考虑为Barrett食管，需要注意哪些并发症？\n回答: '

In [41]:
response, history = model.chat(tokenizer, query=query, history=[])

In [42]:
response

'Barrett食管的并发症包括胃食管反流病、食管出血、溃疡、食管穿孔等。需要及时就医治疗，避免并发症的发生。'

In [50]:
datasets = load_from_disk(dataset_path='data/Med_datasets.jsonl')

In [74]:
datasets['train'][1]

{'context': '指令：下面是一个医学领域的问题，请根据你对医学领域的了解，严谨细致的回答，如果不清楚就回答不清楚，不许胡编乱造。\n问题：患者孕14周，主诉右上腹部疼痛，伴随恶心、呕吐，BP 160/110 mmHg，体检腹肌紧张，有肝区压痛。请问可能患有什么疾病？\n回答: ',
 'target': '患者可能患有妊娠期高血压疾病引起的妊娠期肝血肿及破裂，需要进行B超检查、实验室检查等辅助检查，对症给予肝素、冻干人血浆等治疗方案。'}

In [75]:
query = datasets['train'][1]['context']

In [76]:
query

'指令：下面是一个医学领域的问题，请根据你对医学领域的了解，严谨细致的回答，如果不清楚就回答不清楚，不许胡编乱造。\n问题：患者孕14周，主诉右上腹部疼痛，伴随恶心、呕吐，BP 160/110 mmHg，体检腹肌紧张，有肝区压痛。请问可能患有什么疾病？\n回答: '

In [77]:
response, history = model.chat(tokenizer, query=query, history=[])

In [78]:
response

'可能是妊娠期高血压综合征。建议进行B超检查和胎盘功能检查，以确定诊断。治疗方案包括降压、利尿剂、抗高血压药物、维生素A等。'

In [81]:
model.active_adapter

'LoraAdapter'

In [103]:
model.base_model.model.transformer.encoder.layers[0].self_attention.query_key_value.disable_adapters

False

In [ ]:
model.disable_adapter_layers()